### Amazon User-Profile SequencePattern

This Part is from WTG case to reach the TA by seeing what is their purchasing sequence pattern. We use two part to find out important (high confidence/ high connected) product to Target Product.

#### Phase 1 
- (Optional) Doing the Prefixsapn Module Analysis (only sequence frequent)
- We use `SPADE module to find out the important product`.
- We also Draw the network chart by Gephi(https://gephi.org/) to see the pic.
#### Phase 2
- We add time factor to see `the time these important product direct to Target Product`.


In [204]:
import pandas as pd
from prefixspan import PrefixSpan
import numpy as np
import csv
from pycspade.helpers import spade, print_result
import datetime
import matplotlib.pyplot as plt


### Data Preparing

In [3]:
db = pd.read_csv('M_WTG_AMZ_Profile_All_Data.csv')
db = db.query("user_name !='Amazon Customer'")
# db = db.drop(columns='Unnamed: 0')


##### Optional
1. Uncomment first Cell to do intersection of users data between two or more product
2. Uncomment second Cell to filt the important product 

In [4]:
db2 = pd.read_csv('WTG_AMZ_Profile_All_Data.csv')
db = db.append(db2)


In [ ]:
# important_bdlist = pd.read_csv('output.csv')
# important_bdlist = important_bdlist['breakdown_Category']
# db = db[db['Breakdown_category'].isin(important_bdlist)]


### (Optional) PREFIXSPAN ANALYZE

##### Data Preprocessing
1. With out time Threshold
2. With Threshold

In [ ]:
# 1)
db_Ca = db.query(f"Category == 'ComputersAccessories' ")
db_Ca = db_Ca[['user_name', 'Breakdown_category']]
name_list = list(db_Ca['user_name'])
db_Ca = db_Ca.groupby('user_name')
pro_db = []
for name in name_list:
    try:
        a = db_Ca.get_group(name)['Breakdown_category']
        pro_db.append(a)
    except:
        pass
print('Check{}'.format(pro_db[0]))


In [ ]:
# 2)
def preProcess(db, categoryfilt, dayfilt):
    db_Ca = db.query(f"Category == '{categoryfilt}' ")
    db_Ca = db_Ca[['user_name', 'Breakdown_category', 'review_date_place']]
    db_Ca['review_date_place'] = pd.to_datetime(db_Ca['review_date_place'])
    db_Ca.info()
    name_list = list(db_Ca['user_name'])
    db_Ca = db_Ca.groupby('user_name')
    db_Ca7 = []
    for name in name_list:
        a = db_Ca.get_group(name).reset_index()
        for x in range(len(a['review_date_place'])):
            b = [a.iloc[x]['Breakdown_category']]
            for s in range(len(a['review_date_place'])):
                if x != s:
                    day_gap = (
                        a['review_date_place'][s] - a['review_date_place'][x]
                    ).days
                    if 0 <= abs(day_gap) <= dayfilt:
                        b.append(a.iloc[s]['Breakdown_category'])
            if len(b) != 1:
                db_Ca7.append(b)
    return db_Ca7


# Making Timethershold (Optional)
pro_db = preProcess(db=db, categoryfilt='ComputersAccessories', dayfilt=7)


In [ ]:
def sequencePattern(pro_db, minsup, pattern):
    ps = PrefixSpan(pro_db)
    if pattern == 'frequent':
        result = [
            [freq, set(order)]
            for freq, order in ps.frequent(minsup)
            if len(set(order)) >= 3
        ]
    elif pattern == 'topk':
        result = [
            (freq, set(order)) for freq, order in ps.topk(50) if len(set(order)) >= 3
        ]
    return result


seqP = sequencePattern(pro_db=pro_db, minsup=50, pattern='frequent')
seqP = sorted(seqP, key=lambda tup: tup[0], reverse=True)


In [ ]:
uiq = []
for i in seqP:
    if i[1] not in uiq:
        uiq.append(i[1])
        print(i)


In [ ]:
with open('PatternSequence.csv', 'w') as f:

    # using csv.writer method from CSV package
    write = csv.writer(f)

    write.writerows(seqP)


### SPADE ANALYZE

In [ ]:
db_Ca = db.query(f"Category == 'ComputersAccessories' ")
db_Ca['review_date_place'] = pd.to_datetime(db_Ca['review_date_place'])
db_Ca = (
    db_Ca[['user_name', 'review_date_place', 'Breakdown_category']]
    .sort_values(by=['review_date_place'])
    .reset_index()
)
name_list = list(db_Ca['user_name'].unique())
db_Ca = (
    db_Ca.groupby('user_name')
    .filter(lambda g: any(g.Breakdown_category == 'USBLamps'))
    .groupby('user_name')
)  # including USBLamps
db_Ca = db_Ca.filter(lambda g: any(g.Breakdown_category == 'Monitors')).groupby(
    'user_name'
)
pro_db = []
uid = 0
for name in name_list:
    uid += 1
    try:
        a = db_Ca.get_group(name)[['Breakdown_category', 'review_date_place']]
        pro_db.append(a)
    except:
        pass


In [ ]:
bd_idx = {
    '{}'.format(db['Breakdown_category'].unique()[i]): i
    for i in range(len(db['Breakdown_category'].unique()))
}


In [ ]:
for id in range(len(pro_db)):
    pro_db[id]['Breakdown_category'] = [
        bd_idx.get(i) for i in pro_db[id]['Breakdown_category']
    ]
print(f'Check:{pro_db[0]}')


In [ ]:
uid = 0
spade_list = []
for a in pro_db:
    a['review_date_place'] - pd.to_timedelta(30, unit='d')  #Setting Time Slices Long
    x = a.set_index('review_date_place').groupby(pd.Grouper(freq="1M"))[
        'Breakdown_category'
    ]
    for key in x.groups.keys():
        try:
            month = key.dayofyear // 30
            tmp = [f'{uid}', f'{key.year}{month}', list((x.get_group(key)))]
            spade_list.append(tmp)
        except:
            pass
    uid += 1


In [ ]:
def get_key(val):
    for key, value in bd_idx.items():
        if val == value:
            return key


In [ ]:
# Depend on Data amount Filt by suitable support
result = spade(data=spade_list, support=0.1, parse=True)


In [ ]:
sp_frame = pd.DataFrame(
    {'Occurs': [], 'Accum': [], 'Support': [], 'Confid': [], 'Lift': [], 'Sequence': []}
)


def show_frame(result, result_Frame):
    nseqs = result['nsequences']
    for mined_object in result['mined_objects']:
        conf = 'N/A'
        lift = 'N/A'
        if mined_object.confidence:
            conf = '{:0.7f}'.format(mined_object.confidence)
        if mined_object.lift:
            lift = '{:0.7f}'.format(mined_object.lift)
        result_Frame = result_Frame.append(
            {
                'Occurs': mined_object.noccurs,
                'Accum': mined_object.accum_occurs,
                'Support': mined_object.noccurs / nseqs,
                'Confid': conf,
                'Lift': lift,
                'Sequence': ' -> '.join(
                    list(
                        map(
                            str,
                            [
                                get_key(int(str(i).replace('(', '').replace(')', '')))
                                for i in mined_object.items
                            ],
                        )
                    )
                ),
            },
            ignore_index=True,
        )

    return result_Frame


In [ ]:
sp_frame = show_frame(result, sp_frame)
sp_frame = (
    sp_frame.query("Sequence.str.contains('->')")
    .sort_values('Support', ascending=False)
    .drop_duplicates()
)
sp_frame


In [ ]:
# sp_frame.to_csv('MWinner_sequencePattern_all.csv')


#### Take important product out to do sequence time counting

In [186]:
important_list = [
    'USBLamps',
    'Monitors',
    'USBCables',
    'Keyboards',
    'InternalSolidStateDrives',
    'Mice',
    'MonitorStands',
    'MonitorArms',
    'ComputerSpeakers',
    'Stands',
]

db_important = db[db['Breakdown_category'].isin(important_list)].reset_index(drop=True)

target_product = db_important[
    ['user_name', 'review_date_place', 'Breakdown_category']
].query('Breakdown_category == "USBLamps"')
t_name_list = target_product['user_name'].unique()

timegap_frame = pd.DataFrame(columns=important_list)


In [247]:
db_i = db_important[['user_name', 'review_date_place', 'Breakdown_category']].groupby(
    'user_name'
)
tmp_dic = {key: [] for key in important_list}
for name in t_name_list:
    x = db_i.get_group(name)
    ta = x.query(
        'Breakdown_category == "USBLamps"'
    ).drop_duplicates()  # User Target Time
    for date in ta['review_date_place']:  # Each Time in Target Time
        date = pd.to_datetime(date)
        for p in range(len(x)):  # Read Each Time
            p_time = pd.to_datetime(x.iloc[p]['review_date_place'])
            if (
                p_time < date
                and x.iloc[p]['Breakdown_category'] != 'USBLamps'
                and (date - p_time) <= datetime.timedelta(60)
            ):
                tmp_dic[x.iloc[p]['Breakdown_category']].append(date - p_time)

timegap_frame = pd.DataFrame.from_dict(tmp_dic, orient='index').fillna('NaN').T

timegap_frame = timegap_frame.drop(columns=['USBLamps'])


In [254]:
timegap_frame_info = timegap_frame.describe().astype(str('timedelta64[D]'))
def dayToint(day):
    return(str(day).split(' days')[0])
for i in timegap_frame_info:
    timegap_frame_info[i] = list(map(dayToint,timegap_frame_info[i]))
timegap_frame_info.T


,count,mean,std,min,25%,50%,75%,max
Monitors,90,31,17,1,13,33,46,59
USBCables,38,25,14,3,14,22,31,55
Keyboards,22,25,20,1,8,19,45,59
InternalSolidStateDrives,12,30,17,1,22,28,37,58
Mice,17,26,17,3,10,29,40,60
MonitorStands,21,34,14,2,26,40,43,52
MonitorArms,40,26,19,1,5,28,43,57
ComputerSpeakers,20,30,16,10,19,25,48,60
Stands,62,26,14,2,14,28,34,58


In [255]:
timegap_frame_info.T.to_csv('KeyProduct_timeGap.csv')
